<a href="https://colab.research.google.com/github/JulianCarax01/Fondamenti-web-app/blob/main/PrimaIterazione.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
%%writefile dijkstra_serial.cu
#include <iostream>
#include <vector>
#include <cstdlib>
#include <climits>
#include <cuda.h>
#include <curand_kernel.h>

// Kernel CUDA per aggiornare le distanze
__global__ void dijkstraKernel(int *graph, int *distances, char *visited, int n, int currentNode) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x; // Stabiliamo l'indice generale del thread tramite le formule viste a lezione

    // Verifichiamo che l'indice sia inferiore al numero totale dei nodi e che il nodo non sia già stato visitato,
    // inoltre facciamo un check per verificare che la distanza tra due nodi non sia zero
    if (idx < n && !visited[idx] && graph[currentNode * n + idx] > 0) {
        int newDist = distances[currentNode] + graph[currentNode * n + idx];
        atomicMin(&distances[idx], newDist); // Aggiorniamo la distanza usando atomicMin per evitare condizioni di gara
    }
}

// Funzione host per eseguire Dijkstra in CUDA
void dijkstraCUDA(int *graph, int n, int start) {
    // Allocazione memoria su GPU
    int *d_graph;
    int *d_distances;
    char *d_visited; // Usiamo un tipo char per verificare che il nodo sia già stato verificato o no (non riesco a farlo con il bool)

    // Utilizziamo la malloc per andare ad allocare sulla memoria
    cudaMalloc((void **)&d_graph, n * n * sizeof(int));
    cudaMalloc((void **)&d_distances, n * sizeof(int));
    cudaMalloc((void **)&d_visited, n * sizeof(char));

    // Inizializza distanze e visited
    std::vector<int> distances(n, INT_MAX);
    std::vector<char> visited(n, 0);
    distances[start] = 0;

    // Copia grafico su GPU
    cudaMemcpy(d_graph, graph, n * n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_distances, distances.data(), n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_visited, visited.data(), n * sizeof(char), cudaMemcpyHostToDevice); // Cambiato da bool a char

    // Esegui Dijkstra iterativamente
    for (int i = 0; i < n; ++i) {
        // Trovisamo il nodo non visitato con distanza minima
        int minDist = INT_MAX;
        int currentNode = -1;
        for (int j = 0; j < n; j++) {
            // Controlliamo che il nodo non sia stato visitato e che la distanza sia inferiore a quella nota
            if (!visited[j] && distances[j] < minDist) {
                minDist = distances[j];
                currentNode = j;
            }
        }

        if (currentNode == -1) break; // Se non ci sono più nodi raggiungibili
        visited[currentNode] = 1;

        // Aggiorna le distanze in parallelo
        cudaMemcpy(d_distances, distances.data(), n * sizeof(int), cudaMemcpyHostToDevice);
        cudaMemcpy(d_visited, visited.data(), n * sizeof(char), cudaMemcpyHostToDevice);

        dijkstraKernel<<<(n + 255) / 256, 256>>>(d_graph, d_distances, d_visited, n, currentNode);
        cudaDeviceSynchronize();

        cudaMemcpy(distances.data(), d_distances, n * sizeof(int), cudaMemcpyDeviceToHost);
    }

    // Libera memoria GPU
    cudaFree(d_graph);
    cudaFree(d_distances);
    cudaFree(d_visited);

    // Stampa le distanze
    std::cout << "Distanze dal nodo iniziale:\n";
    for (int i = 0; i < n; ++i) {
        std::cout << "Nodo " << i << ": " << distances[i] << "\n";
    }
}
__global__ void generateGraphKernel(int *graph, int n, int minWeight, int maxWeight, unsigned int seed) {
    int row = blockIdx.y * blockDim.y + threadIdx.y; // Riga della matrice
    int col = blockIdx.x * blockDim.x + threadIdx.x; // Colonna della matrice

    // Verifica che i thread lavorino solo su indici validi
    if (row < n && col < n && row < col) {
        curandState state;
        curand_init(seed, row * n + col, 0, &state);

        // Genera un peso casuale
        int weight = curand(&state) % (maxWeight - minWeight + 1) + minWeight;

        // Aggiorna la matrice (simmetria per grafi non orientati)
        graph[row * n + col] = weight;
        graph[col * n + row] = weight;
    }

    // Assicura che la diagonale sia zero
    if (row < n && col == row) {
        graph[row * n + col] = 0;
    }
}


void generateGraphCUDA(int *graph, int n, int minWeight, int maxWeight) {
    int *d_graph;
    size_t size = n * n * sizeof(int);

    // Allocazione della memoria sulla GPU
    if (cudaMalloc(&d_graph, size) != cudaSuccess) {
        std::cerr << "Errore nell'allocazione della memoria sulla GPU." << std::endl;
        return;
    }
    if (cudaMemset(d_graph, 0, size) != cudaSuccess) {
        std::cerr << "Errore durante l'inizializzazione della memoria sulla GPU." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Configurazione della griglia e dei blocchi
    const int blockSize = 16; // Dimensione del blocco
    dim3 threadsPerBlock(blockSize, blockSize); // Blocchi 16x16 thread
    dim3 blocksPerGrid((n + blockSize - 1) / blockSize, (n + blockSize - 1) / blockSize); // Griglia 2D

    // Lancio del kernel
    generateGraphKernel<<<blocksPerGrid, threadsPerBlock>>>(d_graph, n, minWeight, maxWeight, time(NULL));
    if (cudaDeviceSynchronize() != cudaSuccess) {
        std::cerr << "Errore durante l'esecuzione del kernel." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Copia i dati dalla GPU alla CPU
    if (cudaMemcpy(graph, d_graph, size, cudaMemcpyDeviceToHost) != cudaSuccess) {
        std::cerr << "Errore nella copia dei dati dalla GPU alla CPU." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Libera memoria sulla GPU
    cudaFree(d_graph);
}






int main() {
    int n = 1000; // Numero di nodi
    int start = 0; // Nodo iniziale
    int minWeight = 1;
    int maxWeight = 20;

    // Genera il grafo casuale
    std::vector<int> graph(n * n);
    generateGraphCUDA(graph.data(), n, minWeight, maxWeight);

    // Stampa il grafo
    std::cout << "Grafo generato:\n";
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < n; ++j) {
            std::cout << graph[i * n + j] << " ";
        }
        std::cout << "\n";
    }

    dijkstraCUDA(graph.data(), n, start);

    return 0;
}


Overwriting dijkstra_serial.cu


In [105]:
!nvcc -o dijkstra_serial dijkstra_serial.cu
!./dijkstra_serial
!nvidia-smi


Grafo generato:
0 3 14 20 19 4 19 5 2 13 13 18 14 9 14 13 1 12 3 1 9 1 2 12 5 10 4 15 17 7 17 2 3 18 2 8 8 12 6 8 18 9 7 16 13 14 17 12 20 12 2 11 8 20 9 8 7 7 18 9 5 8 9 13 3 11 4 19 3 1 11 16 13 13 11 1 12 8 9 19 1 17 19 19 20 6 6 16 17 3 9 19 6 9 13 3 14 6 2 2 2 2 4 13 16 2 11 1 1 14 5 8 10 18 8 2 5 14 13 5 2 15 13 3 6 10 6 2 2 20 8 3 6 15 5 19 17 1 15 19 1 3 11 8 13 20 9 18 4 13 20 1 7 15 13 9 2 2 20 3 12 13 20 12 15 12 18 14 8 4 17 9 7 7 16 8 12 2 3 12 7 4 13 7 15 2 11 9 15 1 3 15 19 14 11 9 3 16 7 17 18 16 14 2 12 15 2 18 18 20 4 1 19 7 5 10 10 8 8 2 17 8 2 15 17 12 12 10 20 20 2 5 10 18 10 13 6 8 20 5 17 7 19 6 11 16 15 17 18 20 9 3 1 2 6 20 11 5 1 1 10 6 7 2 20 8 3 17 10 2 9 19 20 5 7 19 15 2 16 5 7 18 10 1 18 20 18 13 4 9 14 3 11 19 16 9 7 20 4 7 8 20 15 20 8 14 8 4 5 8 15 4 5 12 6 4 7 11 1 6 4 17 11 8 15 14 2 10 19 8 13 2 19 16 19 2 18 11 15 18 16 17 15 8 2 10 6 12 4 16 3 11 19 10 16 3 7 12 20 13 17 1 8 20 8 4 14 11 13 14 4 6 14 3 8 14 10 4 18 17 20 16 7 9 7 10 5 18 16 2 12 7

In [37]:
!ncu --set full ./dijkstra_serial

==PROF== Connected to process 14777 (/content/dijkstra_serial)
==PROF== Profiling "generateGraphKernel" - 0: 0%....50%....100% - 31 passes
Grafo generato:
0 5 9 7 5 
5 0 6 2 2 
9 6 0 7 3 
7 2 7 0 10 
5 2 3 10 0 
==PROF== Profiling "dijkstraKernel" - 1: 0%....50%....100% - 31 passes
==PROF== Profiling "dijkstraKernel" - 2: 0%....50%....100% - 31 passes
==PROF== Profiling "dijkstraKernel" - 3: 0%....50%....100% - 31 passes
==PROF== Profiling "dijkstraKernel" - 4: 0%....50%....100% - 31 passes
==PROF== Profiling "dijkstraKernel" - 5: 0%....50%....100% - 31 passes
Distanze dal nodo iniziale:
Nodo 0: 0
Nodo 1: 5
Nodo 2: 8
Nodo 3: 7
Nodo 4: 5
==PROF== Disconnected from process 14777
[14777] dijkstra_serial@127.0.0.1
  generateGraphKernel(int *, int, int, int, unsigned int) (1, 1, 1)x(512, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ------------- ------------
    Metric Name               Metric Unit Metric Value
    ----